# Notebook for testing out code segments


In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import pprint
import matplotlib
import numpy as np
import tensorflow as tf
import tensorflow_models as tfm
import matplotlib.pyplot as plt


from official.core import exp_factory
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

/home/darshk/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/darshk/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
expConfig = exp_factory.get_exp_config('retinanet_resnetfpn_coco')

In [4]:
batch_size = 8
num_classes = 4

HEIGHT, WIDTH = 256, 256
IMG_SIZE = [HEIGHT, WIDTH, 3]

# Backbone config.
expConfig.task.freeze_backbone = False
expConfig.task.annotation_file = ''

# Model config.
expConfig.task.model.input_size = IMG_SIZE
expConfig.task.model.num_classes = num_classes
expConfig.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = expConfig.task.model.num_classes

# Training data config.
expConfig.task.train_data.input_path = "../Dataset/road_coco/records/train-00000-of-00001.tfrecord"
expConfig.task.train_data.dtype = 'float32'
expConfig.task.train_data.global_batch_size = batch_size
expConfig.task.train_data.parser.aug_scale_max = 1.0
expConfig.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
expConfig.task.validation_data.input_path = "../Dataset/road_coco/records/valid-00000-of-00001.tfrecord"
expConfig.task.validation_data.dtype = 'float32'
expConfig.task.validation_data.global_batch_size = batch_size

In [5]:
train_steps = 30000
expConfig.trainer.steps_per_loop = 88 # steps_per_loop = num_of_training_examples // train_batch_size

expConfig.trainer.summary_interval = 88
expConfig.trainer.checkpoint_interval = 500
expConfig.trainer.validation_interval = 110
expConfig.trainer.validation_steps =  22 # validation_steps = num_of_validation_examples // eval_batch_size
expConfig.trainer.train_steps = train_steps
expConfig.trainer.optimizer_config.warmup.linear.warmup_steps = 100
expConfig.trainer.optimizer_config.learning_rate.type = 'cosine'
expConfig.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
expConfig.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.4
expConfig.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.24

In [6]:
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(expConfig.as_dict())

{   'runtime': {   'all_reduce_alg': None,
                   'batchnorm_spatial_persistent': False,
                   'dataset_num_private_threads': None,
                   'default_shard_dim': -1,
                   'distribution_strategy': 'mirrored',
                   'enable_xla': False,
                   'gpu_thread_mode': None,
                   'loss_scale': None,
                   'mixed_precision_dtype': 'bfloat16',
                   'num_cores_per_replica': 1,
                   'num_gpus': 0,
                   'num_packs': 1,
                   'per_gpu_thread_count': 0,
                   'run_eagerly': False,
                   'task_index': -1,
                   'tpu': None,
                   'tpu_enable_xla_dynamic_padder': None,
                   'worker_hosts': None},
    'task': {   'allow_image_summary': False,
                'annotation_file': '',
                'differential_privacy_config': None,
                'export_config': {   'cast_detection_c

In [7]:
if expConfig.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

distStrat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [8]:
with distStrat.scope():
    task = tfm.core.task_factory.get_task(expConfig.task, logging_dir="./train-model/")

In [9]:
for images, labels in task.build_inputs(expConfig.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')


images.shape: (8, 256, 256, 3)  images.dtype: tf.float32
labels.keys: dict_keys(['cls_targets', 'box_targets', 'anchor_boxes', 'cls_weights', 'box_weights', 'image_info'])


In [10]:
categoryIndex = {
	0: {
		'id': 0,
		'name': "speedlimit"
	},
	1: {
		'id': 1,
		'name': "crosswalk"
	},
	2: {
		'id': 2,
		'name': "trafficlight"
	},
	3: {
		'id': 3,
		'name': "stop"
	}
}

tfExDecoder = TfExampleDecoder()

In [11]:
def show_batch(raw_records, num_of_examples):
  plt.figure(figsize=(5, 5))
  use_normalized_coordinates=True
  min_score_thresh = 0.30
  for i, serialized_example in enumerate(raw_records):
    plt.subplot(1, 3, i + 1)
    decoded_tensors = tfExDecoder.decode(serialized_example)
    image = decoded_tensors['image'].numpy().astype('uint8')
    scores = np.ones(shape=(len(decoded_tensors['groundtruth_boxes'])))
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image,
        decoded_tensors['groundtruth_boxes'].numpy(),
        decoded_tensors['groundtruth_classes'].numpy().astype('int'),
        scores,
        category_index=categoryIndex,
        use_normalized_coordinates=use_normalized_coordinates,
        max_boxes_to_draw=200,
        min_score_thresh=min_score_thresh,
        agnostic_mode=False,
        instance_masks=None,
        line_thickness=4)

    plt.imshow(image)
    plt.axis('off')
    plt.title(f'Image-{i+1}')
  plt.show()

In [12]:
buffer_size = 20
num_of_examples = 3

matplotlib.use("TkAgg")

raw_records = tf.data.TFRecordDataset(
    expConfig.task.train_data.input_path).shuffle(
        buffer_size=buffer_size).take(num_of_examples)
show_batch(raw_records, num_of_examples)

In [14]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distStrat,
    task=task,
    mode='train_and_eval',
    params=expConfig,
    model_dir="./train-model/",
    run_post_eval=True)

restoring or initializing model...
restored model from ./train-model/ckpt-26028.
restored from checkpoint: ./train-model/ckpt-26028
train | step:  26028 | training until step 26138...
train | step:  26116 | steps/sec:    3.8 | output: 
    {'box_loss': 0.010357562,
     'cls_loss': 0.7745669,
     'learning_rate': 0.016316248,
     'model_loss': 1.2924452,
     'total_loss': 7.5610614,
     'training_loss': 7.5610614}
train | step:  26138 | steps/sec:    7.8 | output: 
    {'box_loss': 0.010711684,
     'cls_loss': 0.83062387,
     'learning_rate': 0.016134454,
     'model_loss': 1.3662082,
     'total_loss': 7.6233935,
     'training_loss': 7.6233935}
 eval | step:  26138 | running 22 steps of evaluation...
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.65s).
Accumulating evaluation results...
DONE (t=0.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Aver

KeyboardInterrupt: 

In [15]:
%reload_ext tensorboard
%tensorboard --logdir './train-model/'

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 11541.

In [16]:
export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[HEIGHT, WIDTH],
    params=expConfig,
    checkpoint_path=tf.train.latest_checkpoint('./train-model/'),
    export_dir='./export-model/')

INFO:tensorflow:Assets written to: ./export-model/assets


INFO:tensorflow:Assets written to: ./export-model/assets


# Prediction Section

In [17]:
def load_image_into_numpy_array(path):
	image = None
	if(path.startswith('http')):
		response = urlopen(path)
		image_data = response.read()
		image_data = BytesIO(image_data)
		image = Image.open(image_data)
	else:
		image_data = tf.io.gfile.GFile(path, 'rb').read()
		image = Image.open(BytesIO(image_data))

	(im_width, im_height) = image.size
	return np.array(image.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)



def build_inputs_for_object_detection(image, input_image_size):
	image, _ = resize_and_crop_image(
		image,
		input_image_size,
		padded_size=input_image_size,
		aug_scale_min=1.0,
		aug_scale_max=1.0)
	return image

In [18]:
imported = tf.saved_model.load('./export-model/')
model_fn = imported.signatures['serving_default']

In [19]:

testDs = tf.data.TFRecordDataset(
    '../Dataset/road_coco/records/valid-00000-of-00001.tfrecord').take(
        num_of_examples)
show_batch(testDs, num_of_examples)

In [20]:
input_image_size = (HEIGHT, WIDTH)
plt.figure(figsize=(20, 20))
min_score_thresh = 0.30 # Change minimum score for threshold to see all bounding boxes confidences.

for i, serialized_example in enumerate(testDs):
  plt.subplot(1, 3, i+1)
  decoded = tfExDecoder.decode(serialized_example)
  image = build_inputs_for_object_detection(decoded['image'], input_image_size)
  image = tf.expand_dims(image, axis=0)
  image = tf.cast(image, dtype = tf.uint8)
  image_np = image[0].numpy()
  result = model_fn(image)
  print(result)
  visualization_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      result['detection_boxes'][0].numpy(),
      result['detection_classes'][0].numpy().astype(int),
      result['detection_scores'][0].numpy(),
      category_index=categoryIndex,
      use_normalized_coordinates=False,
      max_boxes_to_draw=200,
      min_score_thresh=0.01,
      agnostic_mode=False,
      instance_masks=None,
      line_thickness=4)
  plt.imshow(image_np)
  plt.axis('off')

plt.show()


{'detection_classes': <tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'image_info': <tf.Tensor: shape=(1, 4, 2), dtype=float32, numpy=
array([[[256., 256.],
        [256., 256.],
        [  1.,   1.],
        [  0.,   0.]]], dtype=float32)>, 'num_detections': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([100], dtype=int32)>, 'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[ 31.2781   ,  39.739166 , 251.43723  , 256.       ],
        [190.80336  , 144.30083  , 256.       , 223.6906   ],
        [  0.       , 105.890236 , 167.36792  , 256.       ],
        [  0.       , 156.42574  ,  29.938917 , 203.88